# 🏔️ Strava Dynamic Tables + AI - Complete Demo

This notebook provides a complete end-to-end demo of Snowflake Dynamic Tables with Cortex AI integration.

## What This Does:
1. Creates and populates the ACTIVITIES source table with realistic Strava-like data
2. Builds two Dynamic Tables with AI-powered insights
3. Monitors refresh behavior and analyzes performance metrics
4. Demonstrates real-time data processing with incremental refresh

## Prerequisites:
**Run `00_setup_environment.sql` first** to create database, schemas, role, and warehouse.

## Key Features:
- Real-time activity data streaming simulation
- AI-generated performance insights using Cortex COMPLETE
- Automated ETL with dependency management
- Performance monitoring and optimization tools

---


# Part 1: Data Streaming Setup

Create the source table and simulate real-time activity data streaming.


## Setup: Import Libraries and Set Context


In [ ]:
import time
import random
from datetime import datetime, timedelta
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

session = get_active_session()

print("✅ Libraries imported successfully!")
print(f"📊 Connected to Snowflake: {session.get_current_database()}.{session.get_current_schema()}")


In [ ]:
USE ROLE STRAVA_DEMO_ADMIN;
USE WAREHOUSE STRAVA_DEMO_WH;
USE DATABASE STRAVA_DEMO_SAMPLE;
USE SCHEMA RAW_DATA;


## Create Activities Table


In [ ]:
CREATE TABLE IF NOT EXISTS ACTIVITIES (
    activity_id INTEGER,
    athlete_id INTEGER,
    activity_type VARCHAR(50),
    start_date_local TIMESTAMP_NTZ,
    distance_meters FLOAT,
    moving_time_sec INTEGER,
    elapsed_time_sec INTEGER,
    total_elevation_gain_meters FLOAT,
    map_polyline VARCHAR(500),
    average_heartrate INTEGER
);

SELECT 'Activities table ready!' as status;

## Data Generator Function


In [ ]:
def generate_activity_batch(batch_size=10, starting_id=1000):
    """Generate a batch of realistic Strava activity data"""
    
    activity_types = ['Run', 'Ride', 'Swim', 'Walk', 'Hike']
    activities = []
    
    for i in range(batch_size):
        activity_type = random.choice(activity_types)
        
        if activity_type == 'Run':
            distance = random.uniform(3000, 15000)
            pace_min_per_km = random.uniform(4, 7)
            moving_time = int((distance / 1000) * pace_min_per_km * 60)
            heartrate = random.randint(140, 180)
            elevation = random.uniform(20, 200)
            
        elif activity_type == 'Ride':
            distance = random.uniform(15000, 60000)
            speed_kmh = random.uniform(20, 30)
            moving_time = int((distance / 1000) / speed_kmh * 3600)
            heartrate = random.randint(120, 160)
            elevation = random.uniform(100, 800)
            
        elif activity_type == 'Swim':
            distance = random.uniform(1000, 3000)
            pace_min_per_100m = random.uniform(1.5, 2.5)
            moving_time = int((distance / 100) * pace_min_per_100m * 60)
            heartrate = random.randint(130, 170)
            elevation = 0.0
            
        else:
            distance = random.uniform(2000, 10000)
            pace_min_per_km = random.uniform(10, 15)
            moving_time = int((distance / 1000) * pace_min_per_km * 60)
            heartrate = random.randint(90, 130)
            elevation = random.uniform(50, 400)
        
        elapsed_time = int(moving_time * random.uniform(1.0, 1.3))
        athlete_id = random.randint(1, 100)
        map_polyline = f"polyline_{athlete_id}_{starting_id + i}"
        
        activity = {
            'ACTIVITY_ID': starting_id + i,
            'ATHLETE_ID': athlete_id,
            'ACTIVITY_TYPE': activity_type,
            'START_DATE_LOCAL': datetime.now() - timedelta(minutes=random.randint(0, 60)),
            'DISTANCE_METERS': round(distance, 2),
            'MOVING_TIME_SEC': moving_time,
            'ELAPSED_TIME_SEC': elapsed_time,
            'TOTAL_ELEVATION_GAIN_METERS': round(elevation, 2),
            'MAP_POLYLINE': map_polyline,
            'AVERAGE_HEARTRATE': heartrate
        }
        
        activities.append(activity)
    
    return activities

def stream_activities(duration_minutes=5, batch_size=10, interval_seconds=60):
    """Stream activity data into Snowflake"""
    
    print(f"🚀 Starting data stream...")
    print(f"📊 Config: {batch_size} activities every {interval_seconds} seconds for {duration_minutes} minutes\n")
    
    max_id_result = session.sql("SELECT COALESCE(MAX(activity_id), 0) as max_id FROM RAW_DATA.ACTIVITIES").collect()
    current_id = max_id_result[0]['MAX_ID'] + 1
    
    start_time = time.time()
    end_time = start_time + (duration_minutes * 60)
    batch_count = 0
    total_rows = 0
    
    while time.time() < end_time:
        activities = generate_activity_batch(batch_size, starting_id=current_id)
        df = session.create_dataframe(activities)
        df.write.mode("append").save_as_table("RAW_DATA.ACTIVITIES")
        
        batch_count += 1
        total_rows += batch_size
        current_id += batch_size
        
        elapsed = int(time.time() - start_time)
        remaining = int(end_time - time.time())
        print(f"✅ Batch {batch_count}: {batch_size} activities | Total: {total_rows} | Elapsed: {elapsed}s | Remaining: {remaining}s")
        
        if time.time() < end_time:
            time.sleep(interval_seconds)
    
    print(f"\n🎉 Stream complete! {batch_count} batches, {total_rows} total activities inserted")
    final_count = session.sql("SELECT COUNT(*) as total FROM RAW_DATA.ACTIVITIES").collect()
    print(f"🗂️  Total in table: {final_count[0]['TOTAL']}")

print("✅ Functions ready!")

## Run Initial Data Stream

Generate initial dataset for testing. Adjust parameters as needed.


In [ ]:
stream_activities(
    duration_minutes=0.1,
    batch_size=200,
    interval_seconds=5
)

## Verify Initial Data


In [ ]:
USE SCHEMA RAW_DATA;

SELECT 
    activity_type,
    COUNT(*) as count,
    ROUND(AVG(distance_meters/1000.0), 1) as avg_distance_km,
    ROUND(AVG(moving_time_sec/60.0), 1) as avg_duration_min
FROM ACTIVITIES
GROUP BY activity_type
ORDER BY count DESC;

---

# Part 2: Create Dynamic Tables with AI

Build Dynamic Tables with Cortex AI integration for real-time insights.


## Create Dynamic Tables Schema


In [ ]:
CREATE SCHEMA IF NOT EXISTS STRAVA_DYNAMIC_TABLES;
USE SCHEMA STRAVA_DYNAMIC_TABLES;

SELECT 'Dynamic Tables schema ready!' as STATUS;

## Dynamic Table 1: Activity Intelligence

Real-time activity processing with AI-powered performance insights.

**Features:**
- 1-minute LAG for near real-time processing
- AI-generated performance insights using Cortex COMPLETE
- Calculated metrics (pace, duration)


In [ ]:
CREATE OR REPLACE DYNAMIC TABLE activity_intelligence
    TARGET_LAG = '1 MINUTE'
    WAREHOUSE = STRAVA_DEMO_WH
    REFRESH_MODE = INCREMENTAL
    INITIALIZE = ON_CREATE
AS
    SELECT 
        activity_id,
        athlete_id,
        activity_type,
        start_date_local,
        distance_meters,
        moving_time_sec,
        elapsed_time_sec,
        total_elevation_gain_meters,
        average_heartrate,
        CASE 
            WHEN moving_time_sec > 0 THEN ROUND((distance_meters / 1000.0) / (moving_time_sec / 3600.0), 2)::DECIMAL(10,2)
            ELSE 0::DECIMAL(10,2)
        END as pace_kmh,
        SNOWFLAKE.CORTEX.COMPLETE('llama3-8b',
            'Analyze this ' || activity_type || ' activity: ' ||
            ROUND(distance_meters/1000.0, 1) || 'km in ' || 
            ROUND(moving_time_sec/60.0, 0) || ' minutes. ' ||
            'Pace: ' || ROUND((distance_meters / 1000.0) / (moving_time_sec / 3600.0), 1) || 'km/h. ' ||
            'Provide a brief performance insight in 1-2 sentences.'
        ) as ai_performance_insight
    FROM RAW_DATA.ACTIVITIES
    WHERE start_date_local >= DATEADD('day', -30, start_date_local)
;

## Dynamic Table 2: Athlete Performance Dashboard

Aggregated athlete metrics with AI-powered insights.

**Features:**
- 2-minute LAG for aggregated KPIs (allows incremental refresh)
- Performance tier classification
- AI athlete profiling with training recommendations


In [ ]:
CREATE OR REPLACE DYNAMIC TABLE athlete_performance_dashboard
    TARGET_LAG = '2 MINUTES'
    WAREHOUSE = STRAVA_DEMO_WH
    REFRESH_MODE = INCREMENTAL
    INITIALIZE = ON_CREATE
AS
    SELECT 
        athlete_id,
        COUNT(*) as total_activities_7d,
        ROUND(SUM(distance_meters) / 1000.0, 2)::DECIMAL(10,2) as total_distance_km_7d,
        ROUND(AVG(distance_meters) / 1000.0, 2)::DECIMAL(10,2) as avg_distance_km,
        ROUND(SUM(moving_time_sec) / 3600.0, 2)::DECIMAL(10,2) as total_moving_hours_7d,
        ROUND(AVG(pace_kmh), 2)::DECIMAL(10,2) as avg_pace_kmh,
        ROUND(AVG(average_heartrate), 1)::DECIMAL(5,1) as avg_heartrate,
        SUM(total_elevation_gain_meters)::DECIMAL(10,0) as total_elevation_7d,
        CASE 
            WHEN ROUND(AVG(pace_kmh), 2)::DECIMAL(10,2) > 15 THEN 'High Performer'
            WHEN ROUND(AVG(pace_kmh), 2)::DECIMAL(10,2) > 10 THEN 'Regular Athlete'
            ELSE 'Casual User'
        END as performance_tier,
        SNOWFLAKE.CORTEX.COMPLETE('llama3-8b',
            'Based on this athlete data: ' ||
            total_activities_7d || ' activities, ' ||
            ROUND(total_distance_km_7d, 1) || 'km total distance, ' ||
            ROUND(avg_pace_kmh, 1) || 'km/h average pace, ' ||
            ROUND(avg_heartrate, 0) || ' average heart rate. ' ||
            'Provide a brief athlete profile and training recommendations in 2-3 sentences.'
        ) as ai_athlete_profile,
        MAX(start_date_local) as last_activity_date
    FROM activity_intelligence
    GROUP BY athlete_id
;

## View Created Dynamic Tables


In [ ]:
SHOW DYNAMIC TABLES IN SCHEMA STRAVA_DYNAMIC_TABLES;

In [ ]:
SELECT * FROM activity_intelligence LIMIT 5;

In [ ]:
SELECT * FROM athlete_performance_dashboard LIMIT 5;

## Stream More Data to Watch Refresh Behavior

Now that Dynamic Tables are created, let's stream more data to observe the automatic refresh behavior.


In [ ]:
stream_activities(
    duration_minutes=5,
    batch_size=200,
    interval_seconds=60
)


---

# Part 3: Monitoring & Analytics

Monitor Dynamic Table refresh behavior and analyze performance metrics.


## Dynamic Table Status


In [ ]:
SHOW DYNAMIC TABLES IN SCHEMA STRAVA_DYNAMIC_TABLES;

## Refresh History with Row Changes


In [ ]:
SELECT 
    name,
    refresh_start_time,
    refresh_end_time,
    statistics:numInsertedRows::NUMBER as rows_inserted,
    statistics:numDeletedRows::NUMBER as rows_deleted,
    statistics:numCopiedRows::NUMBER as rows_copied,
    DATEDIFF('second', refresh_start_time, refresh_end_time) as duration_seconds,
    state,
    refresh_action,
    refresh_trigger,
    data_timestamp
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE name IN ('ACTIVITY_INTELLIGENCE', 'ATHLETE_PERFORMANCE_DASHBOARD')
ORDER BY refresh_start_time DESC, name;

## Source Table Statistics


In [ ]:
SELECT 
    COUNT(*) as total_activities,
    COUNT(DISTINCT athlete_id) as unique_athletes,
    MIN(start_date_local) as earliest_activity,
    MAX(start_date_local) as latest_activity,
    COUNT(CASE WHEN start_date_local >= DATEADD('hour', -1, CURRENT_TIMESTAMP()) THEN 1 END) as activities_last_hour
FROM RAW_DATA.ACTIVITIES;

## AI-Generated Insights


In [ ]:
SELECT 
    activity_id,
    athlete_id,
    activity_type,
    ROUND(distance_meters/1000.0, 2) as distance_km,
    ROUND(moving_time_sec/60.0, 1) as duration_min,
    ROUND(pace_kmh, 2) as pace_kmh,
    ai_performance_insight,
    start_date_local
FROM STRAVA_DYNAMIC_TABLES.ACTIVITY_INTELLIGENCE
ORDER BY start_date_local DESC
LIMIT 10;

## AI Athlete Profiles


In [ ]:
SELECT 
    athlete_id,
    total_activities_7d,
    ROUND(total_distance_km_7d, 1) as total_km_7d,
    ROUND(avg_pace_kmh, 1) as avg_pace_kmh,
    ROUND(avg_heartrate, 0) as avg_hr,
    performance_tier,
    ai_athlete_profile,
    last_activity_date
FROM STRAVA_DYNAMIC_TABLES.ATHLETE_PERFORMANCE_DASHBOARD
ORDER BY total_distance_km_7d DESC
LIMIT 10;

## Performance Tier Distribution


In [ ]:
SELECT 
    performance_tier,
    COUNT(*) as athlete_count,
    ROUND(AVG(total_distance_km_7d), 1) as avg_distance_km,
    ROUND(AVG(avg_pace_kmh), 1) as avg_pace,
    ROUND(AVG(total_activities_7d), 1) as avg_activities
FROM STRAVA_DYNAMIC_TABLES.ATHLETE_PERFORMANCE_DASHBOARD
GROUP BY performance_tier
ORDER BY avg_pace DESC;

## Activity Type Breakdown


In [ ]:
SELECT 
    activity_type,
    COUNT(*) as count,
    ROUND(AVG(distance_meters/1000.0), 1) as avg_distance_km,
    ROUND(AVG(moving_time_sec/60.0), 1) as avg_duration_min,
    ROUND(AVG(pace_kmh), 1) as avg_pace_kmh,
    ROUND(AVG(average_heartrate), 0) as avg_heartrate
FROM STRAVA_DYNAMIC_TABLES.ACTIVITY_INTELLIGENCE
GROUP BY activity_type
ORDER BY count DESC;

---

# Part 4: Management & Optimization

Tools for managing LAG settings and controlling costs.


## Modify Dynamic Table LAG (Optional)

Adjust refresh frequency to balance freshness vs. cost. Uncomment desired commands.


In [ ]:
-- More aggressive (faster, more expensive)
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ACTIVITY_INTELLIGENCE SET TARGET_LAG = '1 MINUTE';
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ATHLETE_PERFORMANCE_DASHBOARD SET TARGET_LAG = '3 MINUTES';


In [ ]:
-- Less aggressive (slower, cheaper)
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ACTIVITY_INTELLIGENCE SET TARGET_LAG = '5 MINUTES';
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ATHLETE_PERFORMANCE_DASHBOARD SET TARGET_LAG = '10 MINUTES';


## Suspend/Resume Dynamic Tables

Control costs by suspending when not in use. Uncomment desired commands.


In [ ]:
-- Suspend to save costs
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ACTIVITY_INTELLIGENCE SUSPEND;
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ATHLETE_PERFORMANCE_DASHBOARD SUSPEND;


In [ ]:
-- Resume when ready
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ACTIVITY_INTELLIGENCE RESUME;
-- ALTER DYNAMIC TABLE STRAVA_DYNAMIC_TABLES.ATHLETE_PERFORMANCE_DASHBOARD RESUME;


---

# 🎉 Demo Complete!

You've successfully completed the Strava Dynamic Tables + AI demo!

## What You've Accomplished:
✅ Created source table with realistic activity data  
✅ Built two Dynamic Tables with AI integration  
✅ Monitored refresh behavior and performance  
✅ Analyzed AI-generated insights and athlete profiles  

## Next Steps:
1. **Stream More Data:** Re-run the streaming cell to add more activities
2. **Watch Refreshes:** Monitor how Dynamic Tables auto-refresh
3. **Experiment:** Try different LAG settings
4. **Cleanup:** When done, use **04_cleanup.ipynb** to remove all resources

## Key Takeaways:
- Dynamic Tables automate data transformation pipelines
- AI integration provides intelligent insights at scale
- LAG settings control freshness vs. cost tradeoffs
- No orchestration needed - Snowflake manages dependencies
